### INSERT DATA TO SQL

In [1]:
import os 
from dotenv import load_dotenv 
load_dotenv()

import mysql.connector 
import pandas as pd 

In [2]:
# conectar con mysql
cnx = mysql.connector.connect(
    user=os.getenv("MYSQL_USER"),
    password=os.getenv("MYSQL_PASSWORD"),
    host=os.getenv("MYSQL_HOST"),
    database=os.getenv("MYSQL_DATABASE"),
    autocommit=False
)

cursor = cnx.cursor()

In [3]:
# importar .csv para tener un DataFrame
df_lastfm = pd.read_csv("artistas-lastfm.csv")
df_spotify = pd.read_csv('canciones-spotify.csv')

In [4]:
# Ajustar nombre de columna para que coincida con MySQL
df_lastfm = df_lastfm.rename(columns={"similar_artist": "similar_artists"})

In [5]:
# error en la linea 72 - año solo 2015

df_spotify.head(73)

,id,track_name,artist_name,year,genre,album_type,release_date,popularity
0,0LpHC9mhPAQC98IjXZIrif,The Election of 1800,Daveed Diggs,2015,soundtrack,album,2015-09-25,65
1,6oF8ueLn5hIl4PRp17sxW6,That Would Be Enough,Phillipa Soo,2015,soundtrack,album,2015-09-25,66
2,4eeN8erNIbW2osT0knz5vT,I Know Him,Jonathan Groff,2015,soundtrack,album,2015-09-25,65
3,3lXyAQ0kekAvY5LodpWmUs,Ten Duel Commandments,Lin-Manuel Miranda,2015,soundtrack,album,2015-09-25,67
4,1llkMtLL4vbe0pr3NV2ckg,Let Me Hear,"Fear, and Loathing in Las Vegas",2015,soundtrack,album,2015-09-30,59
...,...,...,...,...,...,...,...,...
68,4XX5uZb9PvTKh8Nm2KSJfk,Once Upon a Time,Toby Fox,2015,soundtrack,album,2015-09-15,56
69,3eQijz4pPT7rb020LqdhdX,Another Medium,Toby Fox,2015,soundtrack,album,2015-09-15,56
70,6ZT36tXEKY2vvTCuNK4fi1,Metal Crusher,Toby Fox,2015,soundtrack,album,2015-09-15,54
71,2ydKgIVZAQXeYLWtxU8DFS,Stay Alive - Reprise,Lin-Manuel Miranda,2015,soundtrack,album,2015-09-25,62


In [6]:
df_lastfm.tail(10)

,artist_name,biography,listeners,playcount,similar_artists
1127,Cubita,https://www.last.fm/music/Cubita,5604,52004,Nuno Ribeiro
1128,Chris Brown,"Christopher Maurice Brown (born May 5, 1989) i...",4694935,154333464,Chris Brown & Tyga
1129,PAAX (Tulum),https://www.last.fm/music/PAAX+(Tulum),41100,283484,Antaares
1130,Simone Vitullo,Simone Vitullo DJ and house music producer for...,23050,108481,Chambord
1131,Duda,There are two groups using the name Duda one f...,7143,78402,Peled
1132,Chop Daily,https://www.last.fm/music/Chop+Daily,77762,620161,Keys the Prince
1133,Praiz,"Praise Ugbede Adejo (born 8 March 1984), bette...",20392,120860,Banky W
1134,Marco Avitabile,https://www.last.fm/music/Marco+Avitabile,1276,3704,Peppe Citarella
1135,Lola Jane,https://www.last.fm/music/Lola+Jane,5905,52454,Archie & Sizzle
1136,Masego,"Micah Davis (born June 8, 1993), better known ...",1035548,20285573,SiR


In [7]:
# NULOS - convertir NULOS de NaN a None (para que SQL acepte los datos)
df_lastfm  = df_lastfm.where(pd.notnull(df_lastfm), None)
df_spotify = df_spotify.where(pd.notnull(df_spotify), None)

In [8]:
# IDEMPOTENCIA - Evitar duplicados: vaciar tablas antes de insertar
# Nota: esto borra los datos, pero mantiene la estructura de las tablas
try:
    cursor.execute("SET FOREIGN_KEY_CHECKS = 0;")
    cursor.execute("TRUNCATE TABLE spotify;")
    cursor.execute("TRUNCATE TABLE lastfm;")
    cnx.commit()
    print("✅ Tablas vaciadas. Listo para insertar sin duplicados.")
finally:
    cursor.execute("SET FOREIGN_KEY_CHECKS = 1;")

✅ Tablas vaciadas. Listo para insertar sin duplicados.


In [9]:
# MySQL - insertar data en la tabla LastFM
sql_lastfm = """
INSERT INTO lastfm (artist_name, biography, listeners, playcount, similar_artists)
VALUES (%s, %s, %s, %s, %s)
"""

# Asegurar el orden de columnas para el INSERT
cols_lastfm = ["artist_name", "biography", "listeners", "playcount", "similar_artists"]

cursor.executemany(sql_lastfm, df_lastfm[cols_lastfm].values.tolist())
cnx.commit()

print(f"✅ Insertadas {cursor.rowcount} filas en 'lastfm'.")


✅ Insertadas 1137 filas en 'lastfm'.


**Cambiar 'release_date' de   año '2015'   para   año completo '2015-01-01**

In [10]:
# Arreglar release_date cuando viene solo con el año (ej: "2015")
df_spotify["release_date"] = df_spotify["release_date"].astype(str)

mask_solo_anio = df_spotify["release_date"].str.len() == 4
df_spotify.loc[mask_solo_anio, "release_date"] = df_spotify.loc[mask_solo_anio, "release_date"] + "-01-01"

# Convertir a fecha (si algún valor es raro, queda como NaT)
df_spotify["release_date"] = pd.to_datetime(df_spotify["release_date"], errors="coerce").dt.date

In [11]:
df_spotify.head(20)

,id,track_name,artist_name,year,genre,album_type,release_date,popularity
0,0LpHC9mhPAQC98IjXZIrif,The Election of 1800,Daveed Diggs,2015,soundtrack,album,2015-09-25,65
1,6oF8ueLn5hIl4PRp17sxW6,That Would Be Enough,Phillipa Soo,2015,soundtrack,album,2015-09-25,66
2,4eeN8erNIbW2osT0knz5vT,I Know Him,Jonathan Groff,2015,soundtrack,album,2015-09-25,65
3,3lXyAQ0kekAvY5LodpWmUs,Ten Duel Commandments,Lin-Manuel Miranda,2015,soundtrack,album,2015-09-25,67
4,1llkMtLL4vbe0pr3NV2ckg,Let Me Hear,"Fear, and Loathing in Las Vegas",2015,soundtrack,album,2015-09-30,59
5,7cAexYA1vH6srKeIuKySud,超絶☆ダイナミック!,Kazuya Yoshii,2015,soundtrack,single,2015-10-07,55
6,1DLfR4MOfLYbV6v3xrmWa8,We Know,Lin-Manuel Miranda,2015,soundtrack,album,2015-09-25,66
7,6SHI6STEW51cQkAXBRpLNj,Your Obedient Servant,Leslie Odom Jr.,2015,soundtrack,album,2015-09-25,66
8,2qFIJT5hjqaNFA1GKwl9me,Take a Break,Phillipa Soo,2015,soundtrack,album,2015-09-25,67
9,733tju3KUeatsbjcTRQ04i,Yorktown (The World Turned Upside Down),Original Broadway Cast of Hamilton,2015,soundtrack,album,2015-09-25,68


In [12]:
# --- 1. Filtrar SPOTIFY para respetar el FOREIGN KEY ---
# Solo dejamos canciones cuyo 'artist_name' existe en la tabla LastFM
df_spotify = df_spotify[df_spotify["artist_name"].isin(df_lastfm["artist_name"])]

print(f"Filtrado por FK completado. Filas restantes en df_spotify: {len(df_spotify)}")


Filtrado por FK completado. Filas restantes en df_spotify: 2680


In [13]:
# --- 2. Identificar duplicados por 'id' ---
df_duplicados = df_spotify[df_spotify.duplicated(subset=['id'], keep=False)]

# --- 3. Conservar solo primeras ocurrencias ---
df_unicos = df_spotify.drop_duplicates(subset=['id'], keep='first')

# --- 4. Mostrar IDs duplicadas descartadas ---
print("--- 🛑 IDs duplicadas descartadas ---")
ids_duplicadas = df_duplicados[~df_duplicados.index.isin(df_unicos.index)]['id'].unique()

if ids_duplicadas.size == 0:
    print("No se encontraron IDs duplicadas.")
else:
    print(f"Se descartaron {len(ids_duplicadas)} IDs duplicadas:")
    for track_id in ids_duplicadas:
        print(f"❌ ID duplicada: {track_id}")

print(f"----------------------------------------------------------")
print(f"Total de filas únicas a insertar: {len(df_unicos)}")


# --- 5. Insertar las canciones ÚNICAS en MySQL ---
sql_spotify = """
INSERT INTO spotify (id, track_name, artist_name, `year`, genre, album_type, release_date, popularity)
VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
"""

# Asegurar el orden de columnas para el INSERT
cols_spotify = ["id", "track_name", "artist_name", "year", "genre", "album_type", "release_date", "popularity"]

try:
    cursor.executemany(sql_spotify, df_unicos[cols_spotify].values.tolist())
    cnx.commit()
    print(f"✅ ÉXITO: Se insertaron {cursor.rowcount} filas únicas en la tabla 'spotify'.")
except Exception as e:
    print(f"❌ ERROR durante la inserción final: {e}")
    cnx.rollback()

--- 🛑 IDs duplicadas descartadas ---
Se descartaron 60 IDs duplicadas:
❌ ID duplicada: 72J8YcwCwa4mEKC5ejTEer
❌ ID duplicada: 3JHJV0ATUoKJT6mZKiAViT
❌ ID duplicada: 0WiZqYM92ppK3M3FdpCTTw
❌ ID duplicada: 255wo3uvC0gORTc3jb8GiY
❌ ID duplicada: 3FesugTZgJ8SiA4mMKkufK
❌ ID duplicada: 6PoMhappYRfHHTmEhki7mg
❌ ID duplicada: 0pDtV1UZNOy1tr7BIvXvb2
❌ ID duplicada: 790egsd71LLOVAxnRIPfZN
❌ ID duplicada: 2jbK4mjePUHe4MBLLKTD4p
❌ ID duplicada: 1cPWDVYZMnxoYjYeqoxZcQ
❌ ID duplicada: 1gqlyWcGelFrxUkU6717Cw
❌ ID duplicada: 6X2ZeyRIkDS2agHBEAj9Av
❌ ID duplicada: 2ZsHTWXlJb6wIMcJAl0GUt
❌ ID duplicada: 22ZyuwMDnaK3JEu3KDRuiD
❌ ID duplicada: 7t7S0VpHgPAl2AqKYbDEy3
❌ ID duplicada: 5RdOAhOquiIseyWtxHmLRr
❌ ID duplicada: 4HrYGKCfEpfJxbXFQfOsFO
❌ ID duplicada: 2CCnb2cYymAvad46etfqLo
❌ ID duplicada: 0UXD0WRnJEdfLQ7kVEGdXw
❌ ID duplicada: 2iQQDh49tRELI79nFrTkM2
❌ ID duplicada: 7Gilv913H0QfaMyGxfebqc
❌ ID duplicada: 7yeli0qRvVyxCeHtZJKWjr
❌ ID duplicada: 2fluT6PPAE7cAuEXQD0QeA
❌ ID duplicada: 7wcuQDBeaFIyMAcN

In [14]:
# Verificar conteo de filas
cursor.execute("SELECT COUNT(*) FROM lastfm;")
print("lastfm:", cursor.fetchone()[0])

cursor.execute("SELECT COUNT(*) FROM spotify;")
print("spotify:", cursor.fetchone()[0])

lastfm: 1137
spotify: 2620


In [15]:
cursor.close()
cnx.close()
print("✅ Conexión cerrada.")

✅ Conexión cerrada.
